In [ ]:
#Don't run the code unless necessary

In [ ]:
import pandas as pd
import numpy as np

portfolios_feat = pd.read_pickle('Info Pública/Funds DB/portfolios_features.pickle')
portfolios_feat.head(10)

In [ ]:
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def insert_text_conversor_error (line):
    line = (re.sub('á', 'ï¿½', line))
    line = (re.sub('é', 'ï¿½', line))
    line = (re.sub('í', 'ï¿½', line))
    line = (re.sub('ó', 'ï¿½', line))
    line = (re.sub('ú', 'ï¿½', line))
    return line

def ClearText1 (line):
    line = (re.sub(u'''<[^>]*>|-rrb-|-lrb-|-rsb-|-lsb-|\.|,|°''', '', line))
    tokenizer = RegexpTokenizer('[a-záéíóúàèìòù0-9ñç˘uˇıńŕýĺ´gśźćńüöëïäčďěňřšťůž]+')
    line = tokenizer.tokenize(line)
    line = ' '.join(line)
    return line

def ClearText2 (line):
    line = (re.sub(u'''<[^>]*>|-rrb-|-lrb-|-rsb-|-lsb-|\.|,|°''', '', line))
    tokenizer = RegexpTokenizer('[a-záéíóúàèìòù0-9ñç˘uˇıńŕýĺ´gśźćńüöëïäčďěňřšťůž ¿½]+')
    line = tokenizer.tokenize(line)
    line = ' '.join(line)
    return line

#Main Code

list_funds = list(portfolios_feat['Fund ID'])

df_similar_funds = pd.DataFrame(data={})
df_similar_funds['Fund ID'] = list_funds

sw = stopwords.words('spanish')

stop_words1 = sw

stop_words2 = [insert_text_conversor_error(i_sw) for i_sw in sw]
    
for i_fund in list_funds:
    similarity = []
    i_invpol = portfolios_feat.loc[portfolios_feat['Fund ID'].isin([i_fund])]['Inv Policy'].values[0].lower()
    
    if 'ï¿½' in i_invpol:
        i_text = 'nook'
    else:
        i_text = 'ok'
    
    for j_fund in list_funds:
        
        j_invpol = portfolios_feat.loc[portfolios_feat['Fund ID'].isin([j_fund])]['Inv Policy'].values[0].lower()
        
        if 'ï¿½' in j_invpol:
            j_text = 'nook'
        else:
            j_text = 'ok'
        
        if i_text == 'ok' and j_text == 'nook':
            
            i_invpol = insert_text_conversor_error(i_invpol)
            i_invpol = ClearText2(i_invpol)
            j_invpol = ClearText2(j_invpol)

            # remove stop words from the string
            X_set = {w for w in i_invpol if not w in stop_words2} 
            Y_set = {w for w in j_invpol if not w in stop_words2}            
            
        elif i_text == 'nook' and j_text == 'ok':
            
            i_invpol = ClearText2(i_invpol)
            j_invpol = insert_text_conversor_error(j_invpol)
            j_invpol = ClearText2(j_invpol)
            
            # remove stop words from the string
            X_set = {w for w in i_invpol if not w in stop_words2} 
            Y_set = {w for w in j_invpol if not w in stop_words2}  
           
        elif i_text == 'nook' and j_text == 'nook':
            
            i_invpol = ClearText2(i_invpol)
            j_invpol = ClearText2(j_invpol)
            
            # remove stop words from the string
            X_set = {w for w in i_invpol if not w in stop_words2} 
            Y_set = {w for w in j_invpol if not w in stop_words2}  
            
        elif i_text == 'ok' and j_text == 'ok':
            
            i_invpol = ClearText1(i_invpol)
            j_invpol = ClearText1(j_invpol)
            
            # remove stop words from the string
            X_set = {w for w in i_invpol if not w in stop_words1} 
            Y_set = {w for w in j_invpol if not w in stop_words1}  

        l1 =[];l2 =[]

        # form a set containing keywords of both strings 
        rvector = X_set.union(Y_set) 
        for w in rvector:
            if w in X_set: l1.append(1) # create a vector
            else: l1.append(0)
            if w in Y_set: l2.append(1)
            else: l2.append(0)
        c = 0

        # cosine formula --> Text similarity
        for i in range(len(rvector)):
                c+= l1[i]*l2[i]
        similarity_cosine = c / float((sum(l1)*sum(l2))**0.5)
        
        similarity += [similarity_cosine]

    df_similar_funds[i_fund] = similarity  
    
df_similar_funds

In [ ]:
df_similar_funds.to_pickle('policy_similarity.pickle')